In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from youtube_transcript_api import YouTubeTranscriptApi

# Youtube scrapping transcript

In [3]:
video_ids = ["oL58Ognb_BY","xXM9ChnjbwU","z_x6PCFblPw","uB0uSQdeZeg"]

In [38]:
transcript_list = []
for video_id in video_ids:
    result = YouTubeTranscriptApi.get_transcript(video_id)
    transcript = ""
    for x in result:
#         transcript = transcript + x["text"] + ". "
        transcript_list.append(x["text"]+".")

# SUD model - text segmentation

In [6]:
# from lstm_for_sud import config
# from lstm_for_sud.dataset import Dataset
# from lstm_for_sud.model import LstmModel
# from lstm_for_sud.utils import load_vocab, get_trimmed_w2v_vectors, Timer

In [7]:
# # initialize models/ instantiate required objects here
# vocab = load_vocab(config.VOCAB_DATA)
# embeddings = get_trimmed_w2v_vectors(config.W2V_DATA)

# MODEL_NAME = "mgb_sud"
# model = LstmModel(embeddings=embeddings)
# model.build()
# model.restore_session(MODEL_NAME)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from data/trained_model/mgb_sud


In [9]:
# data = transcript_list

In [10]:
# words = [[vocab[w] if w in vocab else vocab['$UNK$'] for w in s.strip().split()] for s in data]
# test = Dataset(words=words)

# y_pred = model.predict(test, MODEL_NAME)
# labels = []
# for pred, w in zip(y_pred, test.words):
#     labels.append(list(map(int, pred[:len(w)])))
    
# sentences = []
# for i, label in enumerate(labels):
#     tokens = data[i].split()
#     for j in range(len(label)):
#         if label[j] == 1:
#             tokens[j] += "."
#     sentences.append(" ".join(tokens))

# Spacy - sentence segmentation

Merging sentences in all article into one list

In [12]:
# import spacy
# from spacy.lang.en import English
# import json

In [16]:
# nlp = English()  # just the language with no model
# sentencizer = nlp.create_pipe("sentencizer")
# nlp.add_pipe(sentencizer)
# splited = []
# for article in sentences:
#     doc = nlp(article)
#     for sent in doc.sents:
#         splited.append(str(sent))

# Text Ranking

In [19]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
import tensorflow as tf
import tensorflow_hub as hub

[nltk_data] Downloading package punkt to /home/mxhow/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)

In [21]:
def remove_rubbish(sentence):
    replacedstr = re.sub('\n+', ' ', sentence)
    return replacedstr

In [40]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in transcript_list:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [41]:
sentences[0]

'hello and welcome to this video review.'

In [42]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(sentences))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [43]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(message_embeddings[i].reshape(1,512), message_embeddings[j].reshape(1,512))[0,0]

In [45]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [46]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [48]:
# Extract top 10 sentences as the summary
for i in range(30):
    print(ranked_sentences[i])

(0.0017615984717692977, "image from the camera that's always in.")
(0.0017178164208046134, 'gonna press paper tray oh and then it.')
(0.0016852773910490482, 'just have to take one photo there in.')
(0.0016829551634924495, 'comes with some little plastic like.')
(0.0016800330507357655, "simply pops off and you'll see this.")
(0.0016719558590788735, 'through everything now in your camera.')
(0.0016671064827568404, 'look at it.')
(0.001659670738166662, 'queue up the photos to print and then.')
(0.0016581962190172369, 'loaded the stack of paper into the.')
(0.00165400080928151, 'photo I wanted I just tapped on the.')
(0.0016398404680692533, 'print out a little washed out and you.')
(0.0016394279943124015, 'gonna send it to the little printer here.')
(0.001633918668474628, "it's printing out and then we're gonna.")
(0.0016277986249542266, 'picture works so now the red piece of.')
(0.0016245723964725576, 'through and show you the process of.')
(0.0016241845345878398, "to connect the printer 

In [39]:
len(transcript_list)

824